## **Diffusion Equation**

$$
\frac{d\Sigma}{dt} = \frac{3}{r}\frac{\partial}{\partial r} \left[r^{1/2} \frac{\partial}{\partial r}(\nu \Sigma r^{1/2})\right]
$$

This is evolution equation for the surface density of a geometrically thin disk

Putting this in the form of a diffusion equation:

$$
\frac{\partial f}{\partial t} = D \frac{\partial^2 f}{\partial X^2}
$$
where $X \equiv 2r^{1/2}$, $f \equiv \frac{3}{2}\Sigma X$ and $D = \frac{12\nu}{X^2}$


We need numerical values for $\nu$
$$
\nu = \alpha c_s H \\
H = \frac{c_s}{\Omega} \\
\Omega = \sqrt{\frac{G*M_s}{r^3}} \\
c_s = \sqrt{\frac{k_B * T}{\mu * mH}} \\
mH = \frac{1}{N_A} \\
\mu = 2.3 \\
T = 500K
$$

In [43]:
import numpy as np
import matplotlib.pyplot as plt
import numba

In [44]:
#Natural Constants

T = 600 # Temperature
mu = 2.2 # mean molar mass in disk
avog = 6.02214 * 10**23 # avogadros number
mH = 1.673534 * 10**-27 # atomic hydrogen mass
kB = 1.380649 * 10**-23 #boltzmann constant
G = 6.6738 * 10**-11 #gravitational constant
Ms = 1.9886 * 10**30 #solar mass
Me = 5.972 * 10 **24 #earth mass
AU = 1.4959787 * 10**11 #astronomical units
yrs2sec = 3.1536 * 10**7 #convert years to seconds

In [45]:
#Disk Parameters

alpha = 1.0 * 10**-3
rin = 0.05 * AU #inner radius of the disk
rout = 30 * AU #outer radius of the disk
sigma_in  = 2 * 10**5 #boundary condition at t=0
sigma_max = sigma_in*2 #boundary condition at t=final_time
sigma_min = 1 * 10**2;
distance = rout - rin #distance from inner radius to outer radius

In [46]:
#Temporal discretization

dt = 0.5 * yrs2sec #timestep
final_time = 10*yrs2sec #total diffusion time in seconds

In [47]:
#Spacial discretization

n = 100; #number of steps / number of points to plot
dr = distance/n #distance between each spacial step

In [48]:
#Plotting axes
x = np.linspace(rin + dr/2, rout - dr/2, n)
t = np.arange(0, final_time, dt)
points_range = np.arange(0, len(t)+1, len(t)/n)

In [49]:
#Initialize grid spaces

dist = np.empty(n)
Omega = np.empty(n)
nu = np.empty(n)
X = np.empty(n)
D = np.empty(n)
sigma = np.empty(n)
df_dt = np.empty(n)
f = np.empty(n)
sigma_evol = np.empty(n)

In [50]:
#Boundary Conditions

f_in = 0
f_out = 0

In [51]:
@numba.njit
def calc_init_params(dist, sigma, X, Omega, nu, D, f):
    """
    Calculate the initial parameters values for time t = 0 and changing radius.
    """
    cs2 = (kB * T)/(mu*mH)
    for i in range(n):
        dist[i] = (rin + (rout-rin)*i/(n-1))
        sigma[i] = sigma_in * (AU) / dist[i]
        if (sigma[i]>sigma_max):
            sigma[i] = sigma_max
        if (dist[i]/AU > 15):
            sigma[i] = sigma_min
        X[i] = 2 * np.sqrt(dist[i])
        Omega[i] = np.sqrt(G * Ms / (dist[i] ** 3))
        nu[i] = alpha*cs2/Omega[i]
        D[i] = 12 * nu[i] / (X[i] ** 2)
        f[i] = (1.5 * X[i] * sigma[i])

In [52]:
def plot_time_evol(f, df_dt, dist):
    %matplotlib
    count = 0
    for i in range(1, len(t)+1):
        plt.clf
        for j in range(1, n-1):
            dX = X[j] - X[j-1]
            df_dt[j] = D[j] * (-(f[j] - f[j-1])/dX**2 + (f[j+1]-f[j])/dX**2)
        dX_final = X[-1]-X[-2]
        dX_init = X[1]-X[0]
        df_dt[0] = D[0] * (-(f[0] - f_in)/dX_init**2 + (f[1]-f[0])/dX_init**2)
        df_dt[n-1] = D[n-1] * (-(f[n-1] - f[n-2])/dX_final**2 + (f_out-f[n-1])/dX_final**2)
        f = f + df_dt * dt
        if(i in points_range): #only plot n points 
            sigma_evol = [2*f[k]/(3*X[k]) for k in range(n)]
            mass_evol = [2 * np.pi * dist[k] * dr * sigma_evol[k] for k in range(n)]
            disk_mass = np.sum(mass_evol)
            plt.figure(1)
            plt.plot(x, sigma_evol)
            plt.title('Surface Density Evolution of Protoplanetary Disk')
            plt.xlabel('Radius (m)')
            plt.ylabel('Surface Density')
            plt.xscale('log')
            plt.yscale('log')
            plt.show()
            plt.pause(0.01)
            plt.figure(2)
            plt.plot(x, mass_evol) #expecting a straight line
            plt.title('Conservation of Mass of an Evolving Protoplanetary Disk')
            plt.xlabel('Radius (m)')
            plt.ylabel('Mass')
            plt.xscale('log')
            plt.yscale('log')
    print(f'The mass of the disk is {disk_mass}\n')

In [53]:
calc_init_params(dist, sigma, X, Omega, nu, D, f)

In [54]:
plot_time_evol(f, df_dt, dist)

Using matplotlib backend: MacOSX
The mass of the disk is 4.159328239202436e+29

